# NDVI-based Land Cover Classification
Improved logistic regression model using feature engineering and denoising.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression


In [ ]:

train_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
test_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")


In [ ]:
ndvi_columns = [col for col in train_df.columns if '_N' in col]
X_train_raw = train_df[ndvi_columns]
y_train_raw = train_df['class']
X_test_raw = test_df[ndvi_columns]

le = LabelEncoder()
y_train = le.fit_transform(y_train_raw)

imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_raw), columns=ndvi_columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test_raw), columns=ndvi_columns)

def extract_features(df):
    base = df.copy()
    base['ndvi_mean'] = base.mean(axis=1)
    base['ndvi_std'] = base.std(axis=1)
    base['ndvi_min'] = base.min(axis=1)
    base['ndvi_max'] = base.max(axis=1)
    base['ndvi_range'] = base['ndvi_max'] - base['ndvi_min']
    base['ndvi_early_avg'] = base.iloc[:, -10:].mean(axis=1)
    base['ndvi_late_avg'] = base.iloc[:, :10].mean(axis=1)
    base['ndvi_seasonal_diff'] = base['ndvi_late_avg'] - base['ndvi_early_avg']
    return base

X_train_features = extract_features(X_train_imputed)
X_test_features = extract_features(X_test_imputed)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)
X_test_scaled = scaler.transform(X_test_features)


In [ ]:
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
clf.fit(X_train_scaled, y_train)


In [ ]:
y_test_pred = clf.predict(X_test_scaled)
y_test_labels = le.inverse_transform(y_test_pred)

submission = pd.DataFrame({'ID': test_df['ID'], 'class': y_test_labels})
submission.to_csv("submission.csv", index=False)